In [14]:
from quapack.pyRPE import RobustPhaseEstimation
from quapack.pyRPE.quantum import Q
import numpy as np
import pygsti
%matplotlib inline
import matplotlib.pyplot as plt
import scipy
from pygsti.modelpacks import smq2Q_XYICPHASE
from pygsti.circuits import Circuit
from pygsti.tools import create_elementary_errorgen, create_elementary_errorgen_dual
from scipy.linalg import expm, logm, sqrtm

In [ ]:
PAULIS = [
    np.array([[1, 0], [0, 1]]),
    np.array([[0, 1], [1, 0]]),
    np.array([[0, -1j], [1j, 0]]),
    np.array([[1, 0], [0, -1]])
]

In [15]:
def pauli_operator_2q(a, b):
    return np.kron(Q(a), Q(b))

In [16]:
pauly = pauli_operator_2q(1, 1)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [2]:
#Make the unitary cross-resonance gate.  Target angle is (0,0,pi/2).
def CR_model(tix, iy, iz, tzi, tzx, tzy, tzz):
    return scipy.linalg.expm(-1j/2 * ( tix * pygsti.sigmaix + iy * pygsti.sigmaiy + iz * pygsti.sigmaiz +
            tzi * pygsti.sigmazi + tzy * pygsti.sigmazy + tzz * pygsti.sigmazz + 
            (tzx + np.pi/2) * pygsti.sigmazx))

In [3]:
CR_target = CR_model(0,0,0,0,0,0,0)
noise = 0.1*np.random.randn(7)
CR_noisy = CR_model(*noise)

In [4]:
def post_error_pauli_coefs(noisy_unitary, target_unitary):
    """
    returns an effective unitary
    """

array([-0.1067568 , -0.04928445, -0.05918896, -0.07125827,  0.08343959,
       -0.05482342,  0.00760164])

In [6]:
exp_line_labels = (0,1)#What are the labels for the qubits we'll be doing RPE on?
target_model = smq2Q_XYICPHASE.target_model(qubit_labels=exp_line_labels)
target_model['Gcr',0,1] = pygsti.unitary_to_pauligate(CR_model(0,0,0,0,0,0,0))
noisy_model = target_model.copy()
noisy_model['Gcr',exp_line_labels[0],exp_line_labels[1]] = pygsti.unitary_to_pauligate(
    CR_model(*noise))

In [7]:
def make_rpe_circ(k, germ, prep_fid, meas_fid):
    return Circuit(meas_fid) + Circuit(germ)**k + Circuit(prep_fid)